# LOLC Securities Limited


In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path
import os
from Notebooks.scripts.excel_utils import read_excel_table

## Variable Setup
 in this section the variables for this notebook will be defined, here change these data according to the broker

In [39]:
broker_name = "LOLC Securities Limited"
ref_date = pd.Timestamp('2025-08-12')
broker_code = "LSE"
data_dir_name = "Data"
data_env_var = "LSE_DATA_DIR"


#Password Policy
password_change_time = pd.Timedelta(days=60)



## Data import

In [17]:
project_root = Path.cwd().parent

# Resolve the data base directory
env_base = os.getenv(data_env_var)
data_base = Path(env_base) if env_base else (project_root / data_dir_name)

# Compose broker directory path
broker_dir = data_base / broker_code

# Validate broker directory
if not broker_dir.is_dir():
    raise FileNotFoundError(
        f"Data directory not found: {broker_dir}\n"
        f"- Adjust BROKER (currently '{broker_code}') at the top of this cell, or\n"
        f"- Set {data_env_var} to point to your data base folder."
    )

# Collect Excel files (only .xlsx and .xls)
excel_files = [p for pattern in ("*.xlsx", "*.xls") for p in broker_dir.glob(pattern)]
if not excel_files:
    raise FileNotFoundError(
        f"No Excel files (*.xlsx, *.xls) found in: {broker_dir}\n"
        f"- Ensure files are present, or adjust BROKER/{data_env_var}."
    )

# Read all Excel files into a dict
frames = {p.name: read_excel_table(str(p)) for p in excel_files}


In [19]:
system_user = frames['BBO 1 List of System Users.xls']
advisor_maintenance_act = frames['BBO 2 Advisor Maintenance activities.xlsx']
user_maintenance_act = frames['BBO 3 User Maintenance activities.xlsx']
client_details = frames['BBO 4 Client Details Report.xlsx']
login_history = frames['BBO 5 Login History.xls']
user_management_act = frames['OMS 2 User Management Activities.xlsx']
cds_acc_association = frames['OMS 3 CDS Account Association.xlsx']
advisor_code_mapping = frames['OMS 4 Advisor Code Mapping.xlsx']
password_changes = frames['OMS 5 Password Changes.xlsx']

## System User Data

Check if all the users given in the dataset are unique and there are no missing values.

In [28]:
login_ids = system_user['Login ID']
# Case-insensitive unique count (ignores NA)
unique_ci = login_ids.astype('string').str.strip().str.lower().nunique(dropna=True)

# NA count
na_count = login_ids.isna().sum()

# print(np.shape(system_user))

In [29]:
#Filter out the inactive users
active_users = system_user[system_user['Status'] == 'Active']

In [30]:

last_pass = active_users.copy()

# Parse dates
last_pass['Last Login Date'] = pd.to_datetime(last_pass['Last Login Date'], errors='coerce')
last_pass['Last Password Change Date'] = pd.to_datetime(last_pass['Last Password Change Date'], errors='coerce')

In [40]:
last_pass['time from last login'] = ref_date - last_pass['Last Login Date']
long_last_pass = last_pass[last_pass['time from last login'] > password_change_time]

## Session Data


In [54]:

active_ids = (
    active_users['Login ID']
    .astype('string').str.strip()
    .dropna()
    .unique()
)

logins = login_history.copy()
logins['Login ID'] = logins['Login ID'].astype('string').str.strip()
logins['Login IP'] = logins['Login IP'].astype('string').str.strip()

# Keep only rows for active users
logins = logins[logins['Login ID'].isin(active_ids)]

# Recompute shared IPs among active users only
ip_user_counts = logins.groupby('Login IP')['Login ID'].nunique()
shared_ips = ip_user_counts[ip_user_counts > 1]

# Detailed rows for shared IPs
shared_rows = logins[logins['Login IP'].isin(shared_ips.index)].sort_values(['Login IP', 'Login ID'])

# Summary per shared IP
shared_ip_summary = (
    shared_rows.groupby('Login IP')
    .agg(distinct_users=('Login ID', 'nunique'),
         users=('Login ID', lambda s: sorted(s.dropna().unique().tolist())))
    .reset_index()
)

In [55]:
ids = logins['Login ID'].astype('string').str.strip()
active_mask = ids.isin(active_ids)

# Parse timestamps directly; supports "DD.MM.YYYY" or "DD/MM/YYYY" with optional time
login_time = pd.to_datetime(
    logins['Login Time'].astype('string').str.strip(),
    dayfirst=True, errors='coerce', utc=True
)
logout_time = pd.to_datetime(
    logins['Logout Time'].astype('string').str.strip(),
    dayfirst=True, errors='coerce', utc=True
)

duration = logout_time - login_time
valid = login_time.notna() & logout_time.notna()

sel = active_mask & valid & duration.ge(pd.Timedelta(days=1))

long_sessions = (
    logins.loc[sel, ['Login ID', 'Login IP', 'Login Time', 'Logout Time']]
    .assign(**{'Session Duration': duration[sel]})
)
# ... existing code ...

## Maintenance Activities

In [56]:
#Check if the same person has performed activities on the account
# Filter rows where Action Key contains Login ID
sod_data = user_maintenance_act[user_maintenance_act.apply(
    lambda row: str(row['Login ID']) in str(row['Action Key']), axis=1)]


## Samping

In [58]:
print("Summary of the Findings \n-------------------------------------")
print ("User Accounts: ")
print(f"there are: {unique_ci} unique login IDs, Both active and Inactive")
print(f"There are: {na_count} missing values in the Login ID column.")
print(f"Out of these users, there are {len(active_users)} active users. \n")
print("Password Policy \n-------------------------------------")
print(f"there are {len(long_last_pass)} users who have not changed their password for more than {password_change_time} , refer to the bellow table for more details")
display(long_last_pass)
print("")
print("Session Management \n-------------------------------------")
print(f"There are {len(long_sessions)} sessions longer than 1 day among active users. \nbellow are the details of the sessions")
display(long_sessions)
print("")
print(f"There are cases where the same IP address is used by multiple users. \nThe details of the shared IPs are given below this might show that devices are being shared")
display(shared_ip_summary)
print("")
print("Potential SOD violations \n-------------------------------------")
print(f"There are {len(sod_data)} potential SOD violations where the user has made changes for their own account. \nThe details of the violations are given below")
display(sod_data)


Summary of the Findings 
-------------------------------------
User Accounts: 
there are: 87 unique login IDs, Both active and Inactive
There are: 0 missing values in the Login ID column.
Out of these users, there are 36 active users. 

Password Policy 
-------------------------------------
there are 5 users who have not changed their password for more than 60 days 00:00:00 , refer to the bellow table for more details


,Login ID,Name,Added By,Added Date,Role Name,Group Level,Status,Email Address,Last Modified By,Last Modified Date,Last Password Change Date,Last Login Date,Link Advisor,time from last login
19,POORNA,Mr. P Bandara (OS080946),ruwanthac,01/03/2023,DOC_EXECUTIVE,A,Active,poornaba@lolcsecurities.com,<NA>,12/27/2024,2025-03-06,2025-03-17,YES,148 days
24,ENOKA,Mrs. E JAYAMPATHY (LO000014/LO016962),FAZEEN,04/06/2016,HOF,A,Active,enokaj@lolc.com,<NA>,07/02/2025,2021-12-24,2022-02-03,YES,1286 days
33,LAKSHITHAMAN,Mr. L Manoj (LO013471),ruwanthac,09/15/2023,ERM,A,Active,LakshithaMan@LOLC.com,<NA>,12/27/2024,2025-03-12,2025-03-12,YES,153 days
87,CHANAKA,Mr. Chanaka Sanoj (LO015567),ruwanthac,01/05/2021,ADVISOR,A,Active,ChanakaSan@lolcsecurities.com,<NA>,10/06/2021,2025-02-03,2025-02-24,NO,169 days
91,SRIPATHI,Mr. S Senanayeka (LO021201),SURATH,03/20/2025,ADVISOR,A,Active,sripathis@lolcsecurities.com,<NA>,06/17/2025,2025-04-25,2025-04-25,NO,109 days



Session Management 
-------------------------------------
There are 242 sessions longer than 1 day among active users. 
bellow are the details of the sessions


,Login ID,Login IP,Login Time,Logout Time,Session Duration
330,MADHUSHIKAL,10.106.180.98,01/08/2024 09:54:45,01/10/2024 11:08:50,61 days 01:14:05
352,ROHANA2,10.16.133.52,01/08/2024 14:16:12,05/08/2024 13:34:51,3 days 23:18:39
418,MADHUSHIKAL,10.16.133.47,01/10/2024 13:22:32,20/11/2024 13:44:00,50 days 00:21:28
721,FAZEEN,10.16.133.11,02/04/2025 16:58:38,04/04/2025 10:37:02,1 days 17:38:24
740,DUSHANR,10.16.133.60,02/05/2024 09:40:11,03/05/2024 10:38:11,1 days 00:58:00
...,...,...,...,...,...
21982,POORNA,10.16.133.5,31/01/2025 12:03:36,05/02/2025 09:37:36,4 days 21:34:00
22001,BIMAL,10.106.117.98,31/01/2025 14:30:46,03/02/2025 13:36:26,2 days 23:05:40
22090,ANURUDDHA,192.168.140.4,31/05/2024 14:08:11,03/06/2024 11:15:00,2 days 21:06:49
22125,FAZEEN,10.167.63.9,31/05/2024 17:30:23,04/06/2024 17:29:45,3 days 23:59:22



There are cases where the same IP address is used by multiple users. 
The details of the shared IPs are given below this might show that devices are being shared


,Login IP,distinct_users,users
0,10.101.183.126,2,"[ROHANA, ROHANA2]"
1,10.16.133.11,4,"[FAZEEN, IMESH, NAMINDA, THILINA]"
2,10.16.133.13,2,"[MAHANAMA, SURATH]"
3,10.16.133.15,2,"[DUSHANR, USHAN]"
4,10.16.133.16,3,"[DAMITH, FAZEEN, SHERMALP]"
5,10.16.133.20,5,"[AADIL, LUSHIKA, MADHUSHIKAL, ROHANA, ROHANA2]"
6,10.16.133.23,3,"[DILKIP, RIZMINA, THILINA]"
7,10.16.133.24,2,"[RUWAN, SRIPATHI]"
8,10.16.133.25,2,"[FAZEEN, IMESH]"
9,10.16.133.26,3,"[FAZEEN, MADHUSHIKAL, USHAN]"



Potential SOD violations 
-------------------------------------
There are 2 potential SOD violations where the user has made changes for their own account. 
The details of the violations are given below


,Date,Login ID,Function / Interface,Action,Action Key
150,27/12/2024 1:23:23 PM,FAZEEN,Maintenance >> Users >> Edit Users,Edit,LoginID : FAZEEN Name : ASMONE (LO002344 / BI0...
218,30/04/2024 4:09:51 PM,FAZEEN,Maintenance >> Users >> Edit Users,Edit,LoginID : FAZEEN Name : ASMONE (LO002344 / BO0...


## Findings


* Passwords are in MD5 and Unsalted
* There is a Function called Verify Password that could be exploited for a bruteforce attack

